In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import dotenv
import langchain_huggingface

_ = dotenv.load_dotenv(dotenv.find_dotenv())

repo_id = "deepseek-ai/DeepSeek-V3.2"
llm_model = langchain_huggingface.HuggingFaceEndpoint(
    huggingfacehub_api_token=os.environ['HF_TOKEN'],
    repo_id=repo_id, provider='auto', temperature=0.5
)
llm = langchain_huggingface.ChatHuggingFace(
    llm=llm_model
)

## Prompt Template

In [2]:
from langchain_core import prompts
import time

service_template = """
Translate the text that is delimited by triple backticks into a 
style that is {style}. text: ```{text}```
"""
prompt_template = prompts.ChatPromptTemplate.from_template(
    template=service_template
)

service_style_pirate = """
a polite tone that speaks in English Pirate
"""

service_reply = """
Hey there customer, the warranty does not cover cleaning 
expenses for your kitchen because it's your fault that you 
misused your blender by forgetting to put the lid on before 
starting the blender. Tough luck! See ya!
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

start_time = time.time()
service_response = llm.invoke(input=service_messages)
end_time = time.time() - start_time
print("Duration: %.2f, Response: %s"%(end_time, service_response.content))

Duration: 7.37, Response: Ahoy there, me hearty customer! 'Tis a gentle reminder from yer friendly crew that the warranty be not coverin' the costs o' cleanin' yer galley. 

Alas, the mishap be due to a wee oversight—ye forgot to secure the lid afore settin' yer blender to the high seas. 'Tis a bit o' bad fortune, but the responsibility falls upon yer own shoulders, as the terms o' the warranty be clear on such matters. 

We wish ye smooth sailin' ahead, and may yer future blendin' be ever smooth and tidy! Fare thee well, matey!


## Output Parsers

In [3]:
from langchain_core import prompts
import time

review_template = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product 
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, 
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

prompt_template = prompts.ChatPromptTemplate.from_template(
    template=review_template
)

customer_review = """
This leaf blower is pretty amazing.  It has four settings: 
candle blower, gentle breeze, windy city, and tornado. It 
arrived in two days, just in time for my wife's anniversary 
present. I think my wife liked it so much she was speechless. So 
far I've been the only one using it, and I've been using it 
every other morning to clear the leaves on our lawn. It's 
slightly more expensive than the other leaf blowers out there, 
but I think it's worth it for the extra features.
"""

review_messages = prompt_template.format_messages(
    text=customer_review
)

start_time = time.time()
review_response = llm.invoke(input=review_messages)
end_time = time.time() - start_time
print("Duration: %.2f, Response: %s"%(end_time, review_response.content))

Duration: 3.05, Response: ```json
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


## Agent

In [31]:
import datetime
import pydantic
from langchain import tools, agents

@tools.tool
def get_today_date(text: str) -> str:
    """
    Returns todays date, use this for any questions related to 
    knowing todays date. The input should always be an empty 
    string, and this function will always return todays date - 
    any date mathmatics should occur outside this function.
    """
    return "Today is %s"%(str(datetime.date.today()))

class DateInfo(pydantic.BaseModel):
    date: str

agent = agents.create_agent(
    model=llm,
    tools=[get_today_date],
    # response_format=agents.structured_output.ToolStrategy(DateInfo)
)

response = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "What is the date today?"
    }]
})

In [36]:
response['messages'][-1].content

'Today is **January 4, 2026**.'